In [1]:
# 화면 비율 넓게 설정
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max.colwidth',150)

C:\Users\정승우\AppData\Local\Temp\ipykernel_15264\1083280195.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def crawler(link):
    from bs4 import BeautifulSoup
    import urllib.request
    import re

    #웹브라우져 헤더 추가
    #User-Agent를 조작하는 경우
    hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    req = urllib.request.Request(link, headers = hdr)
    data = urllib.request.urlopen(req).read()
    page = data.decode('utf-8','ignore')
    soup = BeautifulSoup(page, 'html.parser')

    prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
    return prod_items


In [3]:
def cpu_crawler(link):
    prod_items = crawler(link)
    i=0
    partslist = []
    for item in prod_items:
        # print(i,item)
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box--full').get_text().split('/') # 상세정보들
        if link[-6:] == '113973': # intel 소캣 split
            socket = details[0].strip()[5:-1] if details else 'N/A' 
        elif link[-6:] == '113990': # amd 소켓 split
            socket = details[0].strip()[6:-1] if details else 'N/A'
        integrated_graphics = True if '내장그래픽: 탑재' in details else False # 내장그래픽
        # print(i,details)
        for detail in details:
            # print(detail)
            if '메모리 규격' in detail:
                memory_type = str(detail[8:])
            if '시네벤치R23(싱글)' in detail:
                singlebench = detail[13:]
            if '시네벤치R23(멀티)' in detail:
                multibench = detail[13:]
        print([name, price, socket, integrated_graphics, memory_type, singlebench, multibench])
        partslist.append([name, price, socket, integrated_graphics, memory_type, singlebench, multibench])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Socket', 'Integrated Graphics','메모리 타입','시네벤치R23(싱글)','시네벤치R23(멀티)'])
    return result

intel_link ='https://prod.danawa.com/list/?cate=113973'
amd_link = 'https://prod.danawa.com/list/?cate=113990'
intel_cpu = cpu_crawler(intel_link)
amd_cpu = cpu_crawler(amd_link)

['인텔 코어i7-14세대 14700K (랩터레이크 리프레시)', '587,270', '1700', True, 'DDR5, DDR4', '2069', '34818']
['인텔 코어i5-14세대 14400F (랩터레이크 리프레시)', '281,840', '1700', False, 'DDR5, DDR4', '1790', '17804']
['인텔 코어i5-12세대 12400F (엘더레이크)', '153,290', '1700', False, 'DDR5, DDR4', '1605', '12201']
['인텔 코어i9-14세대 14900K (랩터레이크 리프레시)', '813,670', '1700', True, 'DDR5, DDR4', '2277', '40146']
['인텔 코어i5-14세대 14500 (랩터레이크 리프레시)', '346,700', '1700', True, 'DDR5, DDR4', '1934', '22045']
['인텔 코어i5-13세대 13400F (랩터레이크)', '238,670', '1700', False, 'DDR5, DDR4', '1789', '14621']
['인텔 코어i5-13세대 13600K (랩터레이크)', '390,940', '1700', True, 'DDR5, DDR4', '1983', '23380']
['인텔 코어i7-14세대 14700KF (랩터레이크 리프레시)', '549,890', '1700', False, 'DDR5, DDR4', '2145', '35218']
['인텔 코어i7-14세대 14700F (랩터레이크 리프레시)', '520,660', '1700', False, 'DDR5, DDR4', '2091', '32090']
['인텔 코어i3-12세대 12100F (엘더레이크)', '105,655', '1700', False, 'DDR5, DDR4', '1666', '8376']
['인텔 코어i5-12세대 12400 (엘더레이크)', '201,260', '1700', True, 'DDR5, DDR4', '1698', '12396'

In [4]:
intel_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,인텔 코어i7-14세대 14700K (랩터레이크 리프레시),"587,270",1700,True,"DDR5, DDR4",2069,34818
1,인텔 코어i5-14세대 14400F (랩터레이크 리프레시),"281,840",1700,False,"DDR5, DDR4",1790,17804
2,인텔 코어i5-12세대 12400F (엘더레이크),"153,290",1700,False,"DDR5, DDR4",1605,12201
3,인텔 코어i9-14세대 14900K (랩터레이크 리프레시),"813,670",1700,True,"DDR5, DDR4",2277,40146
4,인텔 코어i5-14세대 14500 (랩터레이크 리프레시),"346,700",1700,True,"DDR5, DDR4",1934,22045
5,인텔 코어i5-13세대 13400F (랩터레이크),"238,670",1700,False,"DDR5, DDR4",1789,14621
6,인텔 코어i5-13세대 13600K (랩터레이크),"390,940",1700,True,"DDR5, DDR4",1983,23380
7,인텔 코어i7-14세대 14700KF (랩터레이크 리프레시),"549,890",1700,False,"DDR5, DDR4",2145,35218
8,인텔 코어i7-14세대 14700F (랩터레이크 리프레시),"520,660",1700,False,"DDR5, DDR4",2091,32090
9,인텔 코어i3-12세대 12100F (엘더레이크),"105,655",1700,False,"DDR5, DDR4",1666,8376


In [5]:
amd_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,AMD 라이젠5-5세대 7500F (라파엘),"192,570",AM5,False,DDR5,1827,13943
1,AMD 라이젠7-5세대 7800X3D (라파엘),"524,890",AM5,True,DDR5,1820,18219
2,AMD 라이젠5-4세대 5600 (버미어),"142,000",AM4,False,DDR4,1460,10906
3,AMD 라이젠5-5세대 8600G (피닉스),"262,910",AM5,True,DDR5,1662,13513 \n\n닫기\n\n
4,AMD 라이젠5-4세대 5600G (세잔),"251,690",AM4,True,DDR4,1446,10489
5,AMD 라이젠5 PRO 4650G (르누아르),"101,400",AM4,True,DDR4,1446,10489
6,AMD 라이젠7-4세대 5700X (버미어),"216,980",AM4,False,DDR4,1527,13293
7,AMD 라이젠5-5세대 7600 (라파엘),"253,430",AM5,True,DDR5,1855,14303
8,AMD 라이젠9-5세대 7950X3D (라파엘),"841,880",AM5,True,DDR5,2048,36016
9,AMD 라이젠7-4세대 5700X3D (버미어),"285,390",AM4,False,DDR4,2048,36016


In [6]:
#intel_cpu.to_csv('intel.csv',encoding='cp949', index=False)
intel_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,인텔 코어i7-14세대 14700K (랩터레이크 리프레시),"587,270",1700,True,"DDR5, DDR4",2069,34818
1,인텔 코어i5-14세대 14400F (랩터레이크 리프레시),"281,840",1700,False,"DDR5, DDR4",1790,17804
2,인텔 코어i5-12세대 12400F (엘더레이크),"153,290",1700,False,"DDR5, DDR4",1605,12201
3,인텔 코어i9-14세대 14900K (랩터레이크 리프레시),"813,670",1700,True,"DDR5, DDR4",2277,40146
4,인텔 코어i5-14세대 14500 (랩터레이크 리프레시),"346,700",1700,True,"DDR5, DDR4",1934,22045
5,인텔 코어i5-13세대 13400F (랩터레이크),"238,670",1700,False,"DDR5, DDR4",1789,14621
6,인텔 코어i5-13세대 13600K (랩터레이크),"390,940",1700,True,"DDR5, DDR4",1983,23380
7,인텔 코어i7-14세대 14700KF (랩터레이크 리프레시),"549,890",1700,False,"DDR5, DDR4",2145,35218
8,인텔 코어i7-14세대 14700F (랩터레이크 리프레시),"520,660",1700,False,"DDR5, DDR4",2091,32090
9,인텔 코어i3-12세대 12100F (엘더레이크),"105,655",1700,False,"DDR5, DDR4",1666,8376


In [7]:
#amd_cpu.to_csv('amd.csv',encoding='cp949', index=False)
amd_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,AMD 라이젠5-5세대 7500F (라파엘),"192,570",AM5,False,DDR5,1827,13943
1,AMD 라이젠7-5세대 7800X3D (라파엘),"524,890",AM5,True,DDR5,1820,18219
2,AMD 라이젠5-4세대 5600 (버미어),"142,000",AM4,False,DDR4,1460,10906
3,AMD 라이젠5-5세대 8600G (피닉스),"262,910",AM5,True,DDR5,1662,13513 \n\n닫기\n\n
4,AMD 라이젠5-4세대 5600G (세잔),"251,690",AM4,True,DDR4,1446,10489
5,AMD 라이젠5 PRO 4650G (르누아르),"101,400",AM4,True,DDR4,1446,10489
6,AMD 라이젠7-4세대 5700X (버미어),"216,980",AM4,False,DDR4,1527,13293
7,AMD 라이젠5-5세대 7600 (라파엘),"253,430",AM5,True,DDR5,1855,14303
8,AMD 라이젠9-5세대 7950X3D (라파엘),"841,880",AM5,True,DDR5,2048,36016
9,AMD 라이젠7-4세대 5700X3D (버미어),"285,390",AM4,False,DDR4,2048,36016


In [8]:
# 램 가격 크롤링
ddr4_link = 'https://prod.danawa.com/list/?cate=1131326&searchOption=/innerSearchKeyword='
ddr5_link = 'https://prod.danawa.com/list/?cate=11341201&searchOption=/innerSearchKeyword='

def memory_crawling(link):
    crolled_data = crawler(link)
    # 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
    i=0
    partslist = list()
    for item in crolled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        speed = details[2]
        partslist.append([name, price, speed])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price','Speed'])
    return result
ddr4 = memory_crawling(ddr4_link)
ddr5 = memory_crawling(ddr5_link)

In [9]:
ddr4
#ddr4.to_csv('ddr4.csv',encoding='cp949', index=False)

,Product Name,Price,Speed
0,ESSENCORE KLEVV DDR4-3200 CL22 파인인포,"75,000",3200MHz (PC4-25600)
1,삼성전자 DDR4-3200,"92,270",3200MHz (PC4-25600)
2,삼성전자 DDR4-2666,"43,090",2666MHz (PC4-21300)
3,ESSENCORE KLEVV DDR4-3600 CL18 BOLT XR 패키지 서린,"98,990",3600MHz (PC4-28800)
4,G.SKILL DDR4-3600 CL18 TRIDENT Z RGB 패키지,"222,420",3600MHz (PC4-28800)
5,마이크론 Crucial DDR4-3200 CL22 대원씨티에스,"42,569",3200MHz (PC4-25600)
6,TeamGroup T-Force DDR4-3600 CL18 Delta RGB 화이트 패키지 서린,"240,000",3600MHz (PC4-28800)
7,ESSENCORE KLEVV DDR4-3200 CL16 BOLT X 서린,"44,890",3200MHz (PC4-25600)
8,G.SKILL DDR4-4000 CL18 TRIDENT Z ROYAL 실버 패키지,"168,990",4000MHz (PC4-32000)
9,ESSENCORE KLEVV DDR4-3600 CL18 BOLT X 서린,"50,490",3600MHz (PC4-28800)


In [10]:
ddr5
#ddr5.to_csv('ddr5.csv',encoding='cp949', index=False)

,Product Name,Price,Speed
0,마이크론 Crucial DDR5-5600 CL46 PRO 패키지 대원씨티에스,"376,470",5600MHz (PC5-44800)
1,TeamGroup DDR5-5600 CL46 Elite 서린,"124,000",5600MHz (PC5-44800)
2,삼성전자 DDR5-5600,"134,480",5600MHz (PC5-44800)
3,ESSENCORE KLEVV DDR5-6000 CL30 CRAS V RGB WHITE 패키지 서린,"350,000",6000MHz (PC5-48000)
4,TeamGroup T-CREATE DDR5-5600 CL46 CLASSIC 패키지 서린,"290,000",5600MHz (PC5-44800)
5,마이크론 Crucial DDR5-5600 CL46 대원씨티에스,"125,000",5600MHz (PC5-44800)
6,ESSENCORE KLEVV DDR5-5600 CL46 파인인포,"118,480",5600MHz (PC5-44800)
7,G.SKILL DDR5-6000 CL30 RIPJAWS M5 RGB 화이트 패키지,"338,000",6000MHz (PC5-48000)
8,TeamGroup T-Force DDR5-6000 CL38 Delta RGB 화이트 패키지 서린,"340,000",6000MHz (PC5-48000)
9,SK하이닉스 DDR5-5600,"165,000",5600MHz (PC5-44800)


In [11]:
# 메인보드 크롤링
def board_crawling(link):
    crolled_data = crawler(link)
    i=0
    partslist = list()
    for item in crolled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        # print(i,details)
        for idx in range(len(details)):
            if '메모리' in details[idx]:
                memory_type = str(details[idx][4:])
                memory_speed = str(details[idx+1].split()[0])
                break
        size = details[2].split()[0]
        partslist.append([name, price, memory_type, memory_speed, size])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Memory_Type','Memory_Speed', 'Size'])
    return result
amd_board_link = 'https://prod.danawa.com/list/?cate=1131249'
intel_board_link = 'https://prod.danawa.com/list/?cate=11345282'
amd_board = board_crawling(amd_board_link)
amd_board

,Product Name,Price,Memory_Type,Memory_Speed,Size
0,ASRock B650M PG Lightning 에즈윈,"159,990",DDR5,7200MHz,M-ATX
1,MSI MAG B650M 박격포 WIFI,"249,000",DDR5,7600MHz,M-ATX
2,ASRock B650M Pro RS 대원씨티에스,"179,040",DDR5,7200MHz,M-ATX
3,MSI MAG B650 토마호크 WIFI,"290,870",DDR5,7600MHz,ATX
4,ASRock B650M PG Lightning 대원씨티에스,"150,980",DDR5,7200MHz,M-ATX
5,ASUS TUF Gaming B550M-PLUS STCOM,"156,000",DDR4,4800MHz,M-ATX
6,ASUS PRIME A520M-A II 대원씨티에스,"79,020",DDR4,4866MHz,M-ATX
7,MSI PRO B650M-A WIFI,"189,000",DDR5,7200MHz,M-ATX
8,GIGABYTE B650M K 피씨디렉트,"159,500",DDR5,8000MHz,M-ATX
9,GIGABYTE B650M K 제이씨현,"163,900",DDR5,8000MHz,M-ATX


In [12]:
intel_board = board_crawling(intel_board_link)
intel_board

,Product Name,Price,Memory_Type,Memory_Speed,Size
0,ASUS PRIME H610M-K D4 인텍앤컴퍼니,"97,840",DDR4,3200MHz,M-ATX
1,MSI MAG B760M 박격포 맥스 WIFI,"213,990",DDR5,7000MHz,M-ATX
2,GIGABYTE B760M AORUS ELITE 피씨디렉트,"188,730",DDR5,7800MHz,M-ATX
3,ASUS TUF Gaming B760M-PLUS II 코잇,"198,920",DDR5,7800MHz,M-ATX
4,MSI MAG B760M 박격포 II,"195,000",DDR5,7800MHz,M-ATX
5,GIGABYTE B760M DS3H D4 피씨디렉트,"137,000",DDR4,5333MHz,M-ATX
6,GIGABYTE Z790 AORUS ELITE 피씨디렉트,"337,410",DDR5,7600MHz,ATX
7,BIOSTAR H610MH D4 이엠텍,"87,070",DDR4,3200MHz,M-ATX
8,MSI MAG Z790 토마호크 맥스 WIFI,"369,000",DDR5,7800MHz,ATX
9,GIGABYTE B760M DS3H 제이씨현,"154,950",DDR5,7600MHz,M-ATX


In [13]:
amd_board.to_csv('amd_board.csv',encoding='cp949', index=False)
intel_board.to_csv('intel_board.csv',encoding='cp949', index=False)

In [14]:
import pandas as pd
# GPU 크롤링
def gpu_crawling(link):
    crawled_data = crawler(link)
    i=0
    partslist = list()
    for item in crawled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        for idx in range(len(details)):
            if '정격파워' in details[idx]:
                # power_usage = str(details[idx].split()[1])
                power_type = str(details[idx].split()[1])
            if '가로(길이)' in details[idx]:
                size = str(details[idx].split()[1])
                break
        partslist.append([name, price, power_type, size])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Power_Type', '가로(길이)'])
    return result
gpu_link = 'https://prod.danawa.com/list/?cate=112753'
gpu = gpu_crawling(gpu_link)
gpu

,Product Name,Price,Power_Type,가로(길이)
0,이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB,"418,020",600W,249.9mm
1,ZOTAC GAMING 지포스 RTX 4060 TWIN Edge OC D6 8GB,"414,870",500W,225.3mm
2,이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB,"557,580",650W,249.9mm
3,MSI 지포스 RTX 4060 Ti 벤투스 2X 블랙 OC D6 16GB,"618,950",550W,199mm
4,MSI 지포스 RTX 4070 SUPER 게이밍 X 슬림 D6X 12GB 트라이프로져3,"977,910",650W,307mm
5,MSI 지포스 RTX 4060 벤투스 2X 블랙 OC D6 8GB,"426,990",550W,199mm
6,MSI 지포스 RTX 4070 Ti SUPER 게이밍 X 슬림 D6X 16GB 트라이프로져3,"1,244,080",700W,307mm
7,XFX 라데온 RX 6800 SWFT 319 CORE D6 16GB,"500,200",650W,340mm
8,갤럭시 GALAX 지포스 RTX 4060 WHITE OC D6 8GB,"410,400",550W,237mm
9,이엠텍 지포스 RTX 4060 MIRACLE WHITE D6 8GB,"426,950",600W,252mm


In [15]:
gpu.to_csv('gpu.csv',encoding='cp949', index=False)

In [16]:
# 파워 크롤링

In [17]:
# 다나와 인텔 CPU 가격만 크롤링 코드
from bs4 import BeautifulSoup
import urllib.request
import re

#User-Agent를 조작하는 경우(아이폰에서 사용하는 사파리 브라우져의 헤더)
hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
data ='https://prod.danawa.com/list/?cate=113973'
#웹브라우져 헤더 추가
req = urllib.request.Request(data, \
                            headers = hdr)

data = urllib.request.urlopen(req).read()
page = data.decode('utf-8','ignore')
soup = BeautifulSoup(page, 'html.parser')
crolled_data = soup.find_all(['p','strong'],attrs={'class':'price_sect'})

# 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
i=0
partslist = list()
for item in crolled_data:
    if i>=4 and i!=9:
        # strip 메서드로 앞 뒤 공백 제거
        print(item.select_one('strong').get_text())
    i+=1
print(partslist)

# add_csv('cpu', partslist)

535,990
281,840
277,990
264,680
313,790
130,493
121,850
151,560
813,670
792,860
659,040
346,700
348,990
340,350
398,980
192,570
238,670
224,910
212,650
247,680
390,940
394,430
384,880
420,060
549,890
507,510
612,650
520,660
500,000
509,170
601,400
105,655
93,709
84,469
100,950
201,260
175,120
164,910
283,300
453,410
457,220
447,790
579,900
445,400
424,080
481,380
405,050
369,120
417,550
704,190
716,650
695,670
723,170
692,400
504,290
503,830
494,390
636,160
361,990
341,780
325,670
386,960
156,000
118,830
125,700
180,170
568,330
567,420
539,000
750,240
804,240
740,000
730,000
790,700
208,460
174,000
194,560
257,890
353,220
301,990
288,210
548,700
358,410
286,990
277,240
299,910
777,520
767,090
904,090
407,600
333,260
340,390
327,420
537,090
969,370
319,280
285,730
272,080
379,410
218,130
217,610
72,630
68,990
61,360
298,830
259,990
242,020
243,000
415,690
128,680
118,110
105,172
100,013
204,000
106,990
719729
[]
